In [1]:
# IMPORT LIBRARIES

In [2]:
import pandas as pd
import os

In [3]:
# IMPORT MODULES

In [4]:
os.chdir('/home/ccirelli2/Desktop/Docket-Sheet-Classification/Modules')

import Step1_Module_Ngrams_FreqDist_version4_Ngrams as stp1_Ngram_FD
# Module to Calculate Measurements of Central Tendancy
import Step2_P1_Module_Calculations_Central_Tendancy_version4_Ngrams as stp2_cal_CT
# Module to Calculate Top Words
import Step2_P2_Module_Get_Top_Words_version4_Ngrams as stp2_get_words

In [5]:
# IMPORT FREQ DISTRIBUTION FILES FOR EACH NGRAM TYPE

In [7]:
os.chdir(r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/All_Results/Result_Ngrams')
Nograms = pd.read_excel(r'Docketsheet_FreqDist_Nograms_Average_appearance.xlsx')
Bigrams = pd.read_excel(r'Docketsheet_FreqDist_Bigrams_Average_appearance.xlsx')
Trigrams = pd.read_excel(r'Docketsheet_FreqDist_Trigrams_Average_appearance.xlsx')
Quadgrams = pd.read_excel(r'Docketsheet_FreqDist_Quadgrams_Average_appearance.xlsx')

In [ ]:
# RUN FUNCTION TO GET TOP WORDS

In [8]:
def get_top_words_toggle_methodology(dataframe_freq_distribution, methodology_CT, 
                                     methodology_top_words, write2excel = False, 
                                     destination_location = None, 
                                     Ngram_type = 'Missing_Ngram_type'):
    '''
    INPUTS:
                    a.) dataframe_freq_distribution = The pandas dataframe that represents the freq dist
                        created for each of our 4 ngram types (No, Bi, Tri and Quadgrams)
                    d.) Methodology_CT = type of calculation to be performed (see options below)
                    c.) Methodology_top_words = type of methodology to be performed (see options below)
                    d.) Destination_location = destination to write the file. 
                    e.) Ngram_type = Type of Ngram we are dealing with. 
    
    FUNCTION_I:     Methodology_CT - Calculation of Central Tendencys
        Input       a.) The dataframe containing the word frequency distributions. b.) One of the below cal methodologies. sza
        Options     'CalculationI_homebrew_STDV', 
                    'CalculationII_AVG_not_zero', 
                    'CalculationIII_Correlation_Coefficient'
    
    FUNCTION_II:    Methodology - Top Words
        Input       a.) The dataframe containing the word frequency distribution, b.) One of the below methodologies, 
                    c.) The stage, which is defined within this namespace. 
        Options     'Top15_highest_STDV', 
                    'Top15_highest_COCOEF', 
                    'Top5_highest_STDV_lowest_AVG', 
                    'Top5_highest_STDV_AVG_below_20prct', 
                    'Top5_lowest_STDV_highest_AVG', 
                    'Top5_lowest_COCOEF_highest_AVG'
    '''

    
    # Rename Col[0] of our dataframe to 'Ngrams', which is the column that contains our tuples
    df_rename_cols = dataframe_freq_distribution.rename(index = str, columns = {0: 'Ngrams'})  
       
    # Create a new dataframe to capture the top words and set the index to 0-15.  
    # This is the dataframe that will be returned to our user. 
    df_top_words = pd.DataFrame({}, index = [x for x in range(0,15)])
    
    # Reset df_rename_cols to just the columns 0-12
    if len(df_rename_cols.columns) > 12:                                            # changed from 11 to 12
        df_rename_cols = df_rename_cols.iloc[:, 0:12]                               # changed from [:, 0:12] to [:, 0:12]
        
    # Create an object to keep count of our Life Cycle Stage
    Stage = ''

    # Instead of using the column names to keep count of our position in the code, we use a range function.
    # this is the value to which 100 will be added in the below loop before we sort our columns. 
    for num in range(1,13):                                                         # No change.  We ony want to rotate
                                                                                    # the cols containing stages
               
        # Loop Starts at Life Cycle 1.  Set Condition that when it reaches 11 it stops. 
        if df_rename_cols.columns[1] != 101:                                       # Changed from column[0] to column[1]
            
            # Stage
            Stage = df_rename_cols.columns[1]                                      # changed to columns[1], which is stage 1
            
            # Calculate Measures of Central Tendancy
            df_with_measures_of_CT = stp2_cal_CT.get_Measurements_CentralTendancy(df_rename_cols, 
                                                                                  methodology_CT)
            # Get Top Words
            Top_words_all_calc = stp2_get_words.get_top_words(df_with_measures_of_CT, 
                                                              methodology_top_words, 
                                                              Stage)
            
            # Create a String of the current column name
            Col_name = str(Top_words_all_calc.columns[1])                          # Changed to column[1] from [0]
            
            # Capture all rows (top words) and only the target column of the dataframe. 
            Top_words_target_stage = Top_words_all_calc.iloc[:,0]                  # Changed to iloc[:,1] from [:,0]

            # Append to our dataframe that will be returned to the user the top 5 words for each Life Cycle Stage.  
            df_top_words[Col_name] = Top_words_target_stage
            
            # Reset df_rename_cols to just the columns 0-11
            if len(df_rename_cols.columns) > 12:                                   # Changed to 12 from 11
                df_rename_cols = df_rename_cols.iloc[:, 0:12]                      # Changed to [:,0:12] from [:,0:11]
            
            # Rename the first column to = num + 100.  Num = Original Col1 at each iteration.
            df_col_increased_by100 = df_rename_cols.rename(index = str, columns = {num: num+100})
            
            # With the first column renamed, sort ascending, * but only on a slice of the dataframe that
            # containts our stages.  
            df_rotate = df_col_increased_by100.iloc[:,1:12].sort_index(ascending = True, axis = 1)  # Changed to add a slicer
                                                                                             # so as to only sort the 
                                                                                             # columns containing stages. 
                                                                                             # Not sure if this will work. 
            # Re-insert our Ngram column into the dataframe at position 0. 
            df_rotate.insert(loc = 0, column = 'Ngrams', value = df_rename_cols['Ngrams'])
            # Rename or sorted dataframe to df_rename_cols, which will be entered into our function at the start of the loop. 
            df_rename_cols = df_rotate
    
    # Write to Excel
    if write2excel == True:
        print('Writing dataframe to Excel')
        os.chdir(destination_location)
        File_name = 'TopWords' + '_' + str(Ngram_type) + '_' + methodology_CT + '_' + methodology_top_words
        print('File name => ' + File_name)
        stp1_Ngram_FD.write_to_excel(df_top_words, destination_location, File_name)
        print('Your file has been saved to:  ', destination_location, '\n')
    # Once the list of ngrams is complete, return it to the user.
    
    return df_top_words
            
            

In [10]:
Location = r'/home/ccirelli2/Desktop'

In [10]:
# Run for a single Calculation & Methodology

In [11]:
df_with_measures_CT = get_top_words_toggle_methodology(dataframe_freq_distribution = Quadgrams, 
                                                       methodology_CT = 'CalculationII_AVG_not_zero', 
                                                       methodology_top_words = 'Top15_highest_STDV', 
                                                       write2excel = True, 
                                                       destination_location = Location, 
                                                       Ngram_type = 'Quadgrams')

Writing dataframe to Excel
File name => TopWords_Quadgrams_CalculationII_AVG_not_zero_Top15_highest_STDV
Your file has been saved to:   /home/ccirelli2/Desktop 



In [11]:
# LISTS OF OPTIONS PERTAINING TO STDV

In [12]:
Ngrams = [(Nograms,'Nograms'), (Bigrams, 'Bigrams'), (Trigrams,'Trigrams'), (Quadgrams, 'Quadgrams')]
Calculations_STDV = ['CalculationI_homebrew_STDV', 'CalculationII_AVG_not_zero']
Methodologies_STDV = ['Top15_highest_STDV', 'Top5_highest_STDV_lowest_AVG', 'Top5_highest_STDV_AVG_below_20prct', 
                      'Top5_lowest_STDV_highest_AVG']
Location = r'/home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams'

In [13]:
# RUN CODE OVER ALL OPTIONS

In [14]:
for Ngram_type in Ngrams:
    for Calc in Calculations_STDV:
        for Meth in Methodologies_STDV:
            df_with_measures_CT = get_top_words_toggle_methodology(dataframe_freq_distribution = Ngram_type[0], 
                                                       methodology_CT = Calc, 
                                                       methodology_top_words = Meth, 
                                                       write2excel = True, 
                                                       destination_location = Location, 
                                                       Ngram_type = Ngram_type[1])


Writing dataframe to Excel
File name => TopWords_Nograms_CalculationI_homebrew_STDV_Top15_highest_STDV 

Your file has been saved to:   /home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams
Writing dataframe to Excel
File name => TopWords_Nograms_CalculationI_homebrew_STDV_Top5_highest_STDV_lowest_AVG 

Your file has been saved to:   /home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams
Writing dataframe to Excel
File name => TopWords_Nograms_CalculationI_homebrew_STDV_Top5_highest_STDV_AVG_below_20prct 

Your file has been saved to:   /home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams
Writing dataframe to Excel
File name => TopWords_Nograms_CalculationI_homebrew_STDV_Top5_lowest_STDV_highest_AVG 

Your file has been saved to:   /home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams
Writing dataframe to Excel
File name => TopWords_Nograms_CalculationII_AVG_not_zero_Top15_highest_STDV 

Your file has been saved to:   /home/ccirelli2/Desk

In [15]:
# OPTIONS PERTAINING TO COCOEF (SEPARATED AS THE CALCULATION CAN ONLY BE #3)

In [16]:
Ngrams = [(Nograms,'Nograms'), (Bigrams, 'Bigrams'), (Trigrams,'Trigrams'), (Quadgrams, 'Quadgrams')]
Calculations_STDV = ['CalculationIII_Correlation_Coefficient']
Methodologies_STDV = ['Top15_highest_COCOEF', 'Top5_lowest_COCOEF_highest_AVG']

In [17]:
# RUN CODE OVER ALL OPTIONS

In [18]:
for Ngram_type in Ngrams:
    for Calc in Calculations_STDV:
        for Meth in Methodologies_STDV:
            df_with_measures_CT = get_top_words_toggle_methodology(dataframe_freq_distribution = Ngram_type[0], 
                                                       methodology_CT = Calc, 
                                                       methodology_top_words = Meth, 
                                                       write2excel = True, 
                                                       destination_location = Location, 
                                                       Ngram_type = Ngram_type[1])

Writing dataframe to Excel
File name => TopWords_Nograms_CalculationIII_Correlation_Coefficient_Top15_highest_COCOEF 

Your file has been saved to:   /home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams
Writing dataframe to Excel
File name => TopWords_Nograms_CalculationIII_Correlation_Coefficient_Top5_lowest_COCOEF_highest_AVG 

Your file has been saved to:   /home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams
Writing dataframe to Excel
File name => TopWords_Bigrams_CalculationIII_Correlation_Coefficient_Top15_highest_COCOEF 

Your file has been saved to:   /home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams
Writing dataframe to Excel
File name => TopWords_Bigrams_CalculationIII_Correlation_Coefficient_Top5_lowest_COCOEF_highest_AVG 

Your file has been saved to:   /home/ccirelli2/Desktop/Docket-Sheet-Classification/Result_Ngrams
Writing dataframe to Excel
File name => TopWords_Trigrams_CalculationIII_Correlation_Coefficient_Top15_highest_COCOE

In [ ]:
# CREATE A LIST OF A SINGLE SET OF THE KEY WORDS

In [ ]:
def get_key_word_set(key_word_dataframe):
    key_word_list = []
    for x in key_word_dataframe:
        word_list = list(key_word_dataframe[x])
        [key_word_list.append(x) for x in word_list]
    
    key_word_set = list(set(key_word_list))
    
    df = pd.DataFrame(key_word_set)

    return df

In [ ]:
key_word_set = get_key_word_set(df)

In [ ]:
st1.write_to_excel(key_word_set, 'Key_word_set_4_algorithm_input_03.11.2018_v3')